In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Definir o modelo SIR
def sir(t, y, beta=0.3, gamma=0.1, sigma = 0.1):

    S, E, I, R = y
    N = S + E + I + R  # População total
    
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - gamma * E
    dIdt = gamma * E - sigma * I
    dRdt = sigma * I
    
    return [dSdt, dEdt,  dIdt, dRdt]

# Condições iniciais
S0, E0, I0, R0 = 999, 1, 0, 0  # População inicial
t_span = (0, 1)  # Intervalo de tempo

# Definir malha de referência com passo 0.0001
h_ref = 0.0001
t_common = np.arange(t_span[0], t_span[1] + h_ref, h_ref)
print(f"Malha de referência (h=0.0001): {len(t_common)} pontos")

# Passos de tempo para comparar
steps = [0.8, 0.4, 0.2, 0.1]

# Dicionário para armazenar as soluções
solutions = {}


# Resolver com diferentes passos mas avaliando na malha de referência
for h in steps:
    sol_tmp = solve_ivp(sir, t_span, [S0, E0, I0, R0], t_eval=t_common, method='RK45', max_step=h)
    solutions[f'h={h}'] = sol_tmp
    print(f"Passo h={h}: avaliado em {len(sol_tmp.t)} pontos")

# Solução de referência com passo muito pequeno (0.0001)
sol_ref = solve_ivp(sir, t_span, [S0, E0, I0, R0], t_eval=t_common,  method='RK45', max_step=h_ref)
print(f"Referência (h={h_ref}): {len(sol_ref.t)} pontos")

# Função para calcular erros
def calculate_errors(sol, sol_ref):

    errors = {}

    for i, var in enumerate(['S', 'E', 'I', 'R']):
        diff = sol.y[i] - sol_ref.y[i]
        errors[f'{var}_rmse'] = np.linalg.norm(diff) / np.sqrt(len(diff))
    
    return errors


error_analysis = {}
for name, sol in solutions.items():
    error_analysis[name] = calculate_errors(sol, sol_ref)

# Função para calcular inclinação entre dois pontos consecutivos
def calculate_segment_slopes(step_values, rmse_values):

    slopes = []
    for i in range(len(step_values) - 1):
        h1, h2 = step_values[i], step_values[i + 1]
        e1, e2 = rmse_values[i], rmse_values[i + 1]
        
        # Inclinação no espaço log-log
        slope = (np.log10(e2) - np.log10(e1)) / (np.log10(h2) - np.log10(h1))
        slopes.append(slope)
        
    return slopes

# Plot único: Análise de convergência com inclinações por segmento
plt.figure(figsize=(12, 8))

step_values = [0.8, 0.4, 0.2, 0.1]
rmse_values = [error_analysis[f'h={h}']['I_rmse'] for h in step_values]

# Calcular inclinações dos segmentos
segment_slopes = calculate_segment_slopes(step_values, rmse_values)

# Calcular a inclinação global (regressão linear no espaço log-log)
log_steps = np.log10(step_values)
log_rmse = np.log10(rmse_values)
global_slope = np.polyfit(log_steps, log_rmse, 1)[0]

plt.loglog(step_values, rmse_values, 'bo-', linewidth=2, markersize=8, 
           label=f'inclinação global = {global_slope:.2f}')

# Adicionar anotações com inclinações dos segmentos
for i in range(len(segment_slopes)):
    h_mid = np.sqrt(step_values[i] * step_values[i + 1])  # Média geométrica
    e_mid = np.sqrt(rmse_values[i] * rmse_values[i + 1])  # Média geométrica
    
    plt.annotate(f'α = {segment_slopes[i]:.2f}', 
                xy=(h_mid, e_mid), xytext=(10, 10), 
                textcoords='offset points', fontsize=10,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# Linha de referência para convergência de 2ª ordem
ref_line = rmse_values[-1] * (np.array(step_values) / step_values[-1])**5
plt.loglog(step_values, ref_line, 'r--', alpha=0.7, label='Ordem 5')

# Linha de referência para convergência de 4ª ordem
ref_line_4 = rmse_values[-1] * (np.array(step_values) / step_values[-1])**4
plt.loglog(step_values, ref_line_4, 'g--', alpha=0.7, label='Ordem 4')

plt.xlabel('Passo de Tempo (h)')
plt.ylabel('Erro')
plt.title('Análise de Convergência - Modelo SEIR\n(Inclinações por Segmento)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

for name, errors in error_analysis.items():
    print(f"\n{name}:")
    print(f"  RMSE - S: {errors['S_rmse']:.2e}")
    print(f"  RMSE - E: {errors['E_rmse']:.2e}")
    print(f"  RMSE - I: {errors['I_rmse']:.2e}")
    print(f"  RMSE - R: {errors['R_rmse']:.2e}")

print(f"\nInclinação global da convergência: {global_slope:.2f}")

print("\nInclinações por segmento:")
for i, slope in enumerate(segment_slopes):
    print(f"  Segmento {i+1} (h={step_values[i]:.2f} → h={step_values[i+1]:.2f}): α = {slope:.2f}")